In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math

In [ ]:
df= pd.read_csv("sap_storing_data_hu_project.csv")

In [ ]:
pd.set_option('display.max_columns', None)
df.head()

haalt de data's uithalen dat de Betekenis een vraagteken of nvt staan.

hier zijn wat feature dat we belangrijk vinden:<br>
stm_oorz_code : code van wat de probleem is (weet pas was de aannemer aanwezig is) <br>
stm_sap_melddatum: de datum data melding melding is binnen gekommen <br>
stm_sap_meldtijd : de tijdstip van de melding <br>
stm_geo_mld: geocode waar iets gebuurt is. <br>
stm_sap_meldtijd : de datum dat de aanmener anweizig is （het is binnen 8uur het kan zijn dat dit niet van belang is） <br>
stm_aanntpl_tijd : de tijdmoment dat de aanmener anweizig is <br>
stm_fh_dd : de datum dat de probleem opgelost is <br>
stm_aanntpl_tijd : de tijdmoment dat de probleem opgelost is （het is binnen 8uur het kan zijn dat dit niet van belang is） <br>
stm_techn_mld: Bij welke techniekveld de melding hoort<br>
stm_prioriteit: ?(gekrijgen van docent)<br>
stm_fh_duur: hoelang het duurt van melding tot dat de probleem is opgelost

In [ ]:
df2=df[['stm_oorz_code','stm_sap_melddatum','stm_sap_meldtijd','stm_geo_mld','stm_aanntpl_dd','stm_aanntpl_tijd','stm_fh_dd','stm_fh_tijd','stm_techn_mld','stm_prioriteit','stm_fh_duur','stm_progfh_in_tijd']]

In [ ]:
df2.info()

filter functiehersteltijd met bijhoorde oorzaak code voor de baseline model 

# eerste data preperatie

stm_sap_meldtijd, stm_aanntpl_tijd en stm_fh_tijd is een object dat geschikt is voor model
hieronder woordt het omgezet naar welkste minuut het van de dag is als int

In [ ]:
df2=df2.dropna(subset='stm_aanntpl_tijd')

In [ ]:
df2.info()

In [ ]:
# naar datetime tpye 
df2['stm_sap_meldtijd'] = pd.to_datetime(df2['stm_sap_meldtijd'], format='%H:%M:%S', errors='coerce')
# verandert het h:min naar totale minuten
df2['stm_sap_meldtijd'] = df2['stm_sap_meldtijd'].apply(lambda x: x.hour*60 + x.minute)

df2['stm_aanntpl_tijd'] = pd.to_datetime(df2['stm_aanntpl_tijd'], format='%H:%M:%S', errors='coerce')
df2['stm_aanntpl_tijd'] = df2['stm_aanntpl_tijd'].apply(lambda x: x.hour*60 + x.minute)

df2['stm_fh_tijd'] = pd.to_datetime(df2['stm_fh_tijd'], format='%H:%M:%S', errors='coerce')
df2['stm_fh_tijd'] = df2['stm_fh_tijd'].apply(lambda x: x.hour*60 + x.minute)

df2['stm_progfh_in_tijd'] = pd.to_datetime(df2['stm_progfh_in_tijd'], format='%H:%M:%S', errors='coerce')
df2['stm_progfh_in_tijd'] = df2['stm_progfh_in_tijd'].apply(lambda x: x.hour*60 + x.minute)

# target

voor de traget neem we de tijd neem de functie hersteltijd van waarneer de aanmener aanweizig is tot dat weer gereiden mogen worden.<br>
voor de dat stm_sap_meldtijd 0 is neem we stm_fh_tijd 

In [ ]:
df2['targetherstel'] = np.where(
    df2['stm_fh_tijd'] - df2['stm_aanntpl_tijd']>= 0, # als de stm_fh_tijd-stm_aanntpl_tijd positief is
    df2['stm_fh_tijd'] - df2['stm_aanntpl_tijd'], # dan betekende dat nog op de zefde day is, kun je gelijk van elkaar aftrekken
    df2['stm_fh_tijd'] - df2['stm_aanntpl_tijd']+1440 # als niet op zelfdde dag is hersteld, dus een neagetief uitkomst moet er +1440 om de juiste tijd te krijgen.
)

# data understanding en eerste filtering

in de dataset zijn data dat double voor komen 

In [ ]:
null = df2.loc[df2['targetherstel']==0,['targetherstel']]
vijftien = df2.loc[ (df2['targetherstel'] >0) & (df2['targetherstel']<=15 ),['targetherstel']]
vijftien_halfuur = df2.loc[ (df2['targetherstel'] >15) & (df2['targetherstel']<=30 ),['targetherstel']]
halfuur_uur = df2.loc[( df2['targetherstel']> 30) & (df2['targetherstel']<=60) ,['targetherstel']]
een_twee_uur = df2.loc[ (df2['targetherstel']>60) & (df2['targetherstel']<=120) ,['targetherstel']]
twee_drie_uur = df2.loc[ (df2['targetherstel']>120) & (df2['targetherstel']<=180) ,['targetherstel']]
drie_zes_uur = df2.loc[ (df2['targetherstel']>180) & (df2['targetherstel']<=360) ,['targetherstel']]
zes_acht_uur = df2.loc[ (df2['targetherstel']>360) & (df2['targetherstel']<=540) ,['targetherstel']]
acht_plus = df2.loc[df2['targetherstel']> 540,['targetherstel']]
acht_plus = df2.loc[df2['targetherstel']> 540,['targetherstel']]
print(f'socre null: {len(null)/len(df2)}')
print(f'socre tot 15m: {len(vijftien)/len(df2)}')
print(f'socre 15m tot 30m: {len(vijftien_halfuur)/len(df2)}')
print(f'socre 30m tot 1h: {len(halfuur_uur)/len(df2)}')
print(f'socre 1h tot 2h: {len(een_twee_uur)/len(df2)}')
print(f'socre 2h tot 3h: {len(twee_drie_uur)/len(df2)}')
print(f'socre 3h tot 6h: {len(drie_zes_uur)/len(df2)}')
print(f'socre 6h tot 8h: {len(zes_acht_uur)/len(df2)}')
print(f'socre 8h+: {len(acht_plus)/len(df2)}')

we zien dat dat 20% van de meldingen geen probleem zijn dat de spoor op stil staat <br>
21% dat onder de 15 miuten is en 12% tussen 15-30 min <br>
voor meldingen dat de spoor boven de acht uurr stil staan zij er maar 4%

we dorpen data's dat we niet kunnen op getraint kan

filter data's dat stm_fh_duur onder de 15 min en data's boven de 8 uur,het is niet belangrjik in deze project.<br>
we nemen data's van 15min tot 8uur, zo data de model niet telaag gaat onderschaten.

In [ ]:
df2 = df2.loc[ (df2['targetherstel']>15) & (df2['targetherstel']<=540)]

# tweede data preperatie en tweede filtering

stm_sap_melddatum is een ook object dat geschikt is voor modellen <br>
hieronder woordt het omgezet naar welkste dag het van de jaar is als int <br>

In [ ]:
# naar datetime tpye 
df2['stm_sap_melddatum'] = pd.to_datetime(df2['stm_sap_melddatum'], format='%d/%m/%Y')
# kiijk welkeste dag het van de jaar is
df2['stm_sap_melddatum'] = df2['stm_sap_melddatum'].dt.dayofyear

halt alle doubble data er uit

In [ ]:
df2 = df2.drop_duplicates()

halt alle stuaties dat geen aannemer aanweizig waren

In [ ]:
df2=df2.loc[df2['stm_aanntpl_tijd']!=0]

# deerde data preperatie

verdeelt de tijd per half uur

In [ ]:
# zet alle NaN om in 0 
df2[['stm_sap_meldtijd']]= df2[['stm_sap_meldtijd']].fillna('0')
# zet de type als int
df2['stm_sap_meldtijd']=df2['stm_sap_meldtijd'].astype('int32')

In [ ]:
# zet alle NaN om in 0 
df2[['stm_oorz_code']]= df2[['stm_oorz_code']].fillna('0')
# zet de type als int
df2['stm_oorz_code']=df2['stm_oorz_code'].astype('int32')

In [ ]:
# zet alle NaN om in 0 
df2['stm_prioriteit']=df2['stm_prioriteit'].fillna('0')
# zet de type als int
df2['stm_prioriteit']=df2['stm_prioriteit'].astype('int32')

voor 'stm_prioriteit' dat 8 of 9 zijn haalt we er uit om dat ze langer dan een dag zult duren

In [ ]:
df2=df2.loc[(df2['stm_prioriteit']!=8)&(df2['stm_prioriteit']!=9)]

In [ ]:
df2['targetherstel'] = df2['targetherstel'].apply(lambda x: round(x / 10) * 10)

In [ ]:
#df2.to_csv('newdataset.csv', index=False, encoding='utf-8')

In [ ]:
len(df2)/len(df)

# baseline model

we neem 'stm_progfh_in_tijd' en 'stm_aanntpl_tijd' om de voorspelling hersteltijd te bereken. 

In [ ]:
# zet het data tot welkeste minuut van de dag
df_base = df2[['stm_aanntpl_tijd','targetherstel','stm_progfh_in_tijd']]
df_base = df_base.dropna(subset='stm_progfh_in_tijd')
# hal alle waarde van 0 weg (zijn wat data's dat geen verspelling is gegeven die zijn 0)
df_base = df_base.loc[df_base['stm_progfh_in_tijd']>0]
df_base['stm_progfh_in_tijd'] = df_base['stm_progfh_in_tijd'].astype('int32')

In [ ]:
#berken de voorspeling hersteltijd
df_base['hersteltijd'] = np.where(
    df_base['stm_progfh_in_tijd'] - df_base['stm_aanntpl_tijd']>= 0, # als de stm_fh_tijd-stm_aanntpl_tijd positief is
    df_base['stm_progfh_in_tijd'] - df_base['stm_aanntpl_tijd'], # dan betekende dat nog op de zefde day is, kun je gelijk van elkaar aftrekken
    df_base['stm_progfh_in_tijd'] - df_base['stm_aanntpl_tijd']+1440 # als niet op zelfdde dag is hersteld, dus een neagetief uitkomst moet er +24 om de juiste tijd te krijgen.
)

we nemen de voorspelling hersteltijd en berkenen we de RMSE waarde uit

In [ ]:
list1=df_base['hersteltijd'].tolist()
list2=df_base['targetherstel'].tolist()
result = [(a - b)**2 for a, b in zip(list1, list2)]
math.sqrt(sum(result)/len(list1))

In [ ]:
result = [a > b for a, b in zip(list1, list2)]
sum(result)/len(result)

# model

zet de categorische data om in getalen

In [ ]:
df3=df2
df3 = pd.concat([df3, pd.get_dummies(df2['stm_oorz_code']).astype(int)], axis=1)
df3 = pd.concat([df3, pd.get_dummies(df2['stm_techn_mld']).astype(int)], axis=1)
df3 = pd.concat([df3, pd.get_dummies(df2['stm_prioriteit']).astype(int)], axis=1)
df3 = pd.concat([df3, pd.get_dummies(df2['stm_geo_mld']).astype(int)], axis=1)

df3 = df3.drop(columns=['stm_oorz_code', 'stm_techn_mld','stm_prioriteit','stm_geo_mld'])

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
import joblib

verdeel de feature en target

In [ ]:
target = df3['targetherstel']
feature = df3.drop(columns=['targetherstel','stm_aanntpl_dd','stm_fh_dd','stm_progfh_in_tijd','stm_fh_duur','stm_fh_tijd','stm_progfh_in_tijd'])
feature.columns = feature.columns.astype(str)

feats_train, feats_test, cat_train, cat_test = train_test_split(feature, target, random_state=42)

train de model

In [ ]:
reg = LinearRegression().fit(feats_train, cat_train)

bereken de RMSE waarde van de model

In [ ]:
predictions = reg.predict(feats_train)
cat_train.to_list()
result = [(a - b)**2 for a, b in zip(predictions, cat_train)]
math.sqrt(sum(result)/len(result))

als we de worste case neem de verspellings tijd van de model plus de RMSE waarde, hoeveel procent van de gevallen is gedekt.

In [ ]:
result = [a+84 > b for a, b in zip(predictions, cat_train)]
sum(result)/len(result)

berekende RMSE waarde voor de worst case

In [ ]:
result = [(a + 84 - b)**2 for a, b in zip(predictions, cat_train)]
math.sqrt(sum(result)/len(result))

In [ ]:
joblib.dump(reg, 'LinearRegressionModel.joblib')